In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, tx

### Exercise 1




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_parse_segwit`

In [ ]:
# Exercise 1

reload(tx)
run(tx.TxTest('test_parse_segwit'))

### Exercise 2




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_serialize_segwit`

In [ ]:
# Exercise 2

reload(tx)
run(tx.TxTest('test_serialize_segwit'))

### Exercise 3




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sig_hash_bip143`

In [ ]:
# Exercise 3

reload(tx)
run(tx.TxTest('test_sig_hash_bip143'))

In [ ]:
# example for creating a bech32 address
from ecc import S256Point
from helper import encode_bech32_checksum, encode_varstr
sec_hex = '039d5ca49670cbe4c3bfa84c96a8c87df086c6ea6a24ba6b809c9de234496808d5'
point = S256Point.parse(bytes.fromhex(sec_hex))
h160 = point.hash160()
raw = bytes([0])
raw += encode_varstr(h160)
bech32 = encode_bech32_checksum(raw, testnet=False)
print(bech32)

### Exercise 4
#### Create a testnet bech32 address using your private key from the Session 0

Fill in the spreadsheet with your bech32 address.


In [ ]:
# Exercise 4

from ecc import PrivateKey
from helper import encode_bech32_checksum, encode_varstr, hash256, little_endian_to_int
# use the same passphrase from session 0
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret
private_key = PrivateKey(secret)
# get the public key using the .point property
public_key = private_key.point
# get the hash160 of the point
h160 = public_key.hash160()
# the raw bytes to be encrypted starts with the segwit version (0 or b'\x00)
raw = bytes([0])
# next, add the hash160 using encode_varstr
raw += encode_varstr(h160)
# encode to bech32 using encode_bech32_checksum, remember testnet=True
bech32 = encode_bech32_checksum(raw, testnet=True)
# print the address
print(bech32)

### Exercise 5




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:S256Test:test_bech32_address`

In [ ]:
# Exercise 5

reload(ecc)
run(ecc.S256Test('test_bech32_address'))

In [ ]:
# Example for signing a p2wpkh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
raw_tx_hex = '01000000000101cca99b60e1d687e8faaf93e114114e7b5f6382d9f5d45ffb76ac7472ad7d734c0100000000ffffffff014c400f0000000000160014092ab91b37b4182061d9c01199aaac029f89561f0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
z = tx_obj.sig_hash_bip143(input_index)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
tx_in = tx_obj.tx_ins[input_index]
tx_in.witness = [sig, sec]
print(tx_obj.verify_input(input_index))

### Exercise 6




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sign_p2wpkh`

In [ ]:
# Exercise 6

reload(tx)
run(tx.TxTest('test_sign_p2wpkh'))

In [ ]:
# Example for creating a p2wpkh transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
prev_tx_hex = '0f007db8670c8b22ed64d95c61895d9c8e516ec938f99fbe4973fc0172ef93cf'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 1
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = p2wpkh_script(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
tx_obj.sign_input(0, private_key)
print(tx_obj.serialize().hex())

### Exercise 7

#### Create a p2wpkh spending transaction

You have been sent 0.05 testnet BTC. Send 0.03 to `tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au`
and the change back to your bech32 address.


In [ ]:
# Exercise 7

from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from network import SimpleNode
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
private_key = PrivateKey(little_endian_to_int(hash256(passphrase)))
prev_tx_hex = '94448a601fce6961a5fabbc554068460d979b15bee9531e378fbb458bc644378'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 0
fee = 500
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
target_amount = 3000000
# create the transaction input
tx_in = TxIn(prev_tx, prev_index)
# create an array of tx_outs
tx_outs = []
# decode the target address to get the hash160 of the address
_, _, target_h160 = decode_bech32(target_address)
# create the target script pubkey using p2wpkh_script
target_script_pubkey = p2wpkh_script(target_h160)
# add the target transaction output
tx_outs.append(TxOut(target_amount, target_script_pubkey))
# calculate the change amount, remember you were sent 5000000 sats
change_amount = 5000000 - target_amount - fee
# calculate the hash160 for your private key
change_h160 = private_key.point.hash160()
# create the change script pubkey using p2wpkh_script
change_script_pubkey = p2wpkh_script(change_h160)
tx_outs.append(TxOut(change_amount, change_script_pubkey))
# create the transaction with testnet=True and segwit=True
tx_obj = Tx(1, [tx_in], tx_outs, 0, testnet=True, segwit=True)
# sign the one input with your private key
tx_obj.sign_input(0, private_key)
# print the hex to see what it looks like
print(tx_obj.serialize().hex())

### Exercise 8




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_verify_p2sh_p2wpkh`

In [ ]:
# Exercise 8

reload(tx)
run(tx.TxTest('test_verify_p2sh_p2wpkh'))

In [ ]:
# Example of generating a p2sh-p2wpkh address
from ecc import S256Point
from helper import encode_base58_checksum, hash160, h160_to_p2sh_address
from script import p2wpkh_script
sec_hex = '02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d'
point = S256Point.parse(bytes.fromhex(sec_hex))
h160 = point.hash160()
redeem_script = p2wpkh_script(h160)
h160_p2sh = hash160(redeem_script.raw_serialize())
address = h160_to_p2sh_address(h160_p2sh, testnet=False)
print(address)

### Exercise 9
#### Create a testnet p2sh-p2wpkh address with your private key.



In [ ]:
# Exercise 9

from ecc import PrivateKey
from helper import encode_varstr, h160_to_p2sh_address, hash160, hash256, little_endian_to_int
from script import p2wpkh_script
# use the same passphrase from session 0
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret
private_key = PrivateKey(secret)
# get the public key using the .point property
public_key = private_key.point
# get the hash160 of the point
h160 = public_key.hash160()
# create the RedeemScript, which is the p2wpkh_script of the hash160
redeem_script = p2wpkh_script(h160)
# perform a hash160 on the raw serialization of the RedeemScript
p2sh_h160 = hash160(redeem_script.raw_serialize())
# encode to base58 using h160_to_p2sh_address, remember testnet=True
address = h160_to_p2sh_address(p2sh_h160, testnet=True)
# print the address
print(address)

### Exercise 10




#### Make [this test](/edit/session1/script.py) pass: `script.py:ScriptTest:test_p2sh_address`

In [ ]:
# Exercise 10

reload(script)
run(script.ScriptTest('test_p2sh_address'))

### Exercise 11




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:S256Test:test_p2sh_p2wpkh_address`

In [ ]:
# Exercise 11

reload(ecc)
run(ecc.S256Test('test_p2sh_p2wpkh_address'))

In [ ]:
# Example for signing a p2sh-p2wpkh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from script import Script
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
raw_tx_hex = '010000000001014e6b786f3cd70ab1ffd75caa6bb252c9888fdca9ca94d40fec24bec3e643d89e0000000000ffffffff014c400f0000000000160014401af0b57c7a4b7490c508a47d0747d03cf6ac2e0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
z = tx_obj.sig_hash_bip143(input_index, redeem_script=redeem_script)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
tx_in = tx_obj.tx_ins[input_index]
tx_in.witness = [sig, sec]
tx_in.script_sig = Script([redeem_script.raw_serialize()])
print(tx_obj.verify_input(input_index))

### Exercise 12




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sign_p2sh_p2wpkh`

In [ ]:
# Exercise 12

reload(tx)
run(tx.TxTest('test_sign_p2sh_p2wpkh'))

In [ ]:
# Example for creating a p2sh-p2wpkh transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import p2wpkh_script
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
prev_tx_hex = '6c14a8370da20c7de5ebf216ece3156e99e7d6070442d93b80cdc344b2e80867'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 1
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = p2wpkh_script(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
tx_obj.sign_input(0, private_key, redeem_script=redeem_script)
print(tx_obj.serialize().hex())